In [ ]:
#@title Google Colab запретил пользовательский интерфейс Gradio для бесплатных пользователей, но вы все равно можете использовать RVC. Обучение пока ТОЛЬКО, скоро добавлю Inference.

In [ ]:
#@title Установка на Гугл Диск (Для автоматической тренировки и сохранения прогресса.)
from google.colab import drive
from IPython.utils import capture
from IPython.display import clear_output
import ipywidgets as widgets
import os

%cd /content/
drive.mount('/content/drive')
if not os.path.exists('/content/drive'):
    print("Ты не смонтировал свой диск. Сделаем фейк!")
    os.makedirs('/content/drive/MyDrive')
else:
    print("Молодец! Продолжаем.")
with capture.capture_output() as cap:
    def inf(msg, style, wdth): inf = widgets.Button(description=msg, disabled=True, button_style=style, layout=widgets.Layout(min_width=wdth));display(inf)
    !wget https://huggingface.co/NeuroDonu/rvc/resolve/main/project-main.zip 
    !unzip -n 'project-main.zip' -d /content/drive/MyDrive
    %cd /content/drive/MyDrive/project-main
    !python download_files.py 
    %pip install -r requirements-safe.txt
    !rm /content/project-main.zip
    !rm -r /content/sample_data
    !mkdir -p /content/dataset
clear_output()
inf('\u2714 Готово','success', '50px')

In [ ]:
#@title 1.Препроцесс данных
from IPython.utils import capture
from IPython.display import clear_output
import ipywidgets as widgets
import os
def inf(msg, style, wdth): inf = widgets.Button(description=msg, disabled=True, button_style=style, layout=widgets.Layout(min_width=wdth));display(inf)
%cd /content/drive/MyDrive/project-main
model_name = 'My-Voice' #@param {type:"string"}
#@markdown <small> Вставь сюда путь к своему датасету, либо открой проводник тут и загрузи папку с названием 'dataset'.
dataset_folder = '/content/dataset' #@param {type:"string"}
while len(os.listdir(dataset_folder)) < 1:
    input("Твой датасет пустой!.")
!mkdir -p ./logs/{model_name}
with open(f'./logs/{model_name}/preprocess.log','w') as f:
    print("Начинаю...")
!python infer/modules/train/preprocess.py {dataset_folder} 40000 2 ./logs/{model_name} False 3.0 > /dev/null 2>&1
with open(f'./logs/{model_name}/preprocess.log','r') as f:
    if 'end preprocess' in f.read():
        clear_output()
        inf('\u2714 Готово','success', '50px')
    else:
        print("Ошибка препроцесса... Сделай корректный датасет.")

In [ ]:
#@title 2.Извлечение
f0method = "rmvpe_gpu" # @param ["pm", "harvest", "rmvpe", "rmvpe_gpu"]
%cd /content/drive/MyDrive/project-main
with open(f'./logs/{model_name}/extract_f0_feature.log','w') as f:
    print("Starting...")
if f0method != "rmvpe_gpu":
    !python infer/modules/train/extract/extract_f0_print.py ./logs/{model_name} 2 {f0method}
else:
    !python infer/modules/train/extract/extract_f0_rmvpe.py 1 0 0 ./logs/{model_name} True
!python infer/modules/train/extract_feature_print.py cuda:0 1 0 0 ./logs/{model_name} v2
with open(f'./logs/{model_name}/extract_f0_feature.log','r') as f:
    if 'all-feature-done' in f.read():
        clear_output()
        inf('\u2714 Готово','success', '50px')
    else:
        print("Ошибка препроцесса...")

In [ ]:
#@title 3.Тренировка индекса
import numpy as np
import faiss
%cd /content/drive/MyDrive/project-main
def train_index(exp_dir1, version19):
    exp_dir = "logs/%s" % (exp_dir1)
    os.makedirs(exp_dir, exist_ok=True)
    feature_dir = (
        "%s/3_feature256" % (exp_dir)
        if version19 == "v1"
        else "%s/3_feature768" % (exp_dir)
    )
    if not os.path.exists(feature_dir):
        return "请先进行特征提取!"
    listdir_res = list(os.listdir(feature_dir))
    if len(listdir_res) == 0:
        return "请先进行特征提取！"
    infos = []
    npys = []
    for name in sorted(listdir_res):
        phone = np.load("%s/%s" % (feature_dir, name))
        npys.append(phone)
    big_npy = np.concatenate(npys, 0)
    big_npy_idx = np.arange(big_npy.shape[0])
    np.random.shuffle(big_npy_idx)
    big_npy = big_npy[big_npy_idx]
    if big_npy.shape[0] > 2e5:
        infos.append("Trying doing kmeans %s shape to 10k centers." % big_npy.shape[0])
        yield "\n".join(infos)
        try:
            big_npy = (
                MiniBatchKMeans(
                    n_clusters=10000,
                    verbose=True,
                    batch_size=256 * config.n_cpu,
                    compute_labels=False,
                    init="random",
                )
                .fit(big_npy)
                .cluster_centers_
            )
        except:
            info = traceback.format_exc()
            logger.info(info)
            infos.append(info)
            yield "\n".join(infos)

    np.save("%s/total_fea.npy" % exp_dir, big_npy)
    n_ivf = min(int(16 * np.sqrt(big_npy.shape[0])), big_npy.shape[0] // 39)
    infos.append("%s,%s" % (big_npy.shape, n_ivf))
    yield "\n".join(infos)
    index = faiss.index_factory(256 if version19 == "v1" else 768, "IVF%s,Flat" % n_ivf)
    infos.append("training")
    yield "\n".join(infos)
    index_ivf = faiss.extract_index_ivf(index)  #
    index_ivf.nprobe = 1
    index.train(big_npy)
    faiss.write_index(
        index,
        "%s/trained_IVF%s_Flat_nprobe_%s_%s_%s.index"
        % (exp_dir, n_ivf, index_ivf.nprobe, exp_dir1, version19),
    )

    infos.append("adding")
    yield "\n".join(infos)
    batch_size_add = 8192
    for i in range(0, big_npy.shape[0], batch_size_add):
        index.add(big_npy[i : i + batch_size_add])
    faiss.write_index(
        index,
        "%s/added_IVF%s_Flat_nprobe_%s_%s_%s.index"
        % (exp_dir, n_ivf, index_ivf.nprobe, exp_dir1, version19),
    )
    infos.append(
        "成功构建索引，added_IVF%s_Flat_nprobe_%s_%s_%s.index"
        % (n_ivf, index_ivf.nprobe, exp_dir1, version19)
    )

training_log = train_index(model_name, 'v2')

for line in training_log:
    print(line)
    if 'adding' in line:
        clear_output()
        inf('\u2714 Готово','success', '50px')
    else:
        print("Ошибка! Твой индекс не получился")

In [ ]:
#@title 4.Тренировка модели
%cd /content/drive/MyDrive/project-main
from random import shuffle
import json
import os
import pathlib
from subprocess import Popen, PIPE, STDOUT
now_dir=os.getcwd()
#@markdown <small> Введите название вашей модели еще раз. Это должно быть то же самое, что вы выбрали раньше.
model_name = 'My-Voice'#@param {type:"string"}
#@markdown <small> Выберите, как часто сохранять модель и сколько времени вы хотите для ее обучения.
save_frequency = 20 # @param {type:"slider", min:5, max:50, step:5}
epochs = 300 # @param {type:"slider", min:10, max:1000, step:10}
#@markdown <small> Кэшируются ТОЛЬКО наборы данных продолжительностью менее 10 минут. В противном случае оставьте это без флажка.
cache = True #@param {type:"boolean"}

def click_train(
    exp_dir1,
    sr2,
    if_f0_3,
    spk_id5,
    save_epoch10,
    total_epoch11,
    batch_size12,
    if_save_latest13,
    pretrained_G14,
    pretrained_D15,
    gpus16,
    if_cache_gpu17,
    if_save_every_weights18,
    version19,
):
    exp_dir = "%s/logs/%s" % (now_dir, exp_dir1)
    os.makedirs(exp_dir, exist_ok=True)
    gt_wavs_dir = "%s/0_gt_wavs" % (exp_dir)
    feature_dir = (
        "%s/3_feature256" % (exp_dir)
        if version19 == "v1"
        else "%s/3_feature768" % (exp_dir)
    )
    if if_f0_3:
        f0_dir = "%s/2a_f0" % (exp_dir)
        f0nsf_dir = "%s/2b-f0nsf" % (exp_dir)
        names = (
            set([name.split(".")[0] for name in os.listdir(gt_wavs_dir)])
            & set([name.split(".")[0] for name in os.listdir(feature_dir)])
            & set([name.split(".")[0] for name in os.listdir(f0_dir)])
            & set([name.split(".")[0] for name in os.listdir(f0nsf_dir)])
        )
    else:
        names = set([name.split(".")[0] for name in os.listdir(gt_wavs_dir)]) & set(
            [name.split(".")[0] for name in os.listdir(feature_dir)]
        )
    opt = []
    for name in names:
        if if_f0_3:
            opt.append(
                "%s/%s.wav|%s/%s.npy|%s/%s.wav.npy|%s/%s.wav.npy|%s"
                % (
                    gt_wavs_dir.replace("\\", "\\\\"),
                    name,
                    feature_dir.replace("\\", "\\\\"),
                    name,
                    f0_dir.replace("\\", "\\\\"),
                    name,
                    f0nsf_dir.replace("\\", "\\\\"),
                    name,
                    spk_id5,
                )
            )
        else:
            opt.append(
                "%s/%s.wav|%s/%s.npy|%s"
                % (
                    gt_wavs_dir.replace("\\", "\\\\"),
                    name,
                    feature_dir.replace("\\", "\\\\"),
                    name,
                    spk_id5,
                )
            )
    fea_dim = 256 if version19 == "v1" else 768
    if if_f0_3:
        for _ in range(2):
            opt.append(
                "%s/logs/mute/0_gt_wavs/mute%s.wav|%s/logs/mute/3_feature%s/mute.npy|%s/logs/mute/2a_f0/mute.wav.npy|%s/logs/mute/2b-f0nsf/mute.wav.npy|%s"
                % (now_dir, sr2, now_dir, fea_dim, now_dir, now_dir, spk_id5)
            )
    else:
        for _ in range(2):
            opt.append(
                "%s/logs/mute/0_gt_wavs/mute%s.wav|%s/logs/mute/3_feature%s/mute.npy|%s"
                % (now_dir, sr2, now_dir, fea_dim, spk_id5)
            )
    shuffle(opt)
    with open("%s/filelist.txt" % exp_dir, "w") as f:
        f.write("\n".join(opt))

    print("Готово")
    print("Использую видеокарту:", str(gpus16))
    if pretrained_G14 == "":
        print("Нет предварительно обученного генератора")
    if pretrained_D15 == "":
        print("Нет предварительно обученного дискриминатора")
    if version19 == "v2" or sr2 == "40k":
        config_path = "configs/v2/%s.json" % sr2
    else:
        config_path = "configs/v1/%s.json" % sr2
    config_save_path = os.path.join(exp_dir, "config.json")
    if not pathlib.Path(config_save_path).exists():
        with open(config_save_path, "w", encoding="utf-8") as f:
            with open(config_path, "r") as config_file:
                config_data = json.load(config_file)
                json.dump(
                    config_data,
                    f,
                    ensure_ascii=False,
                    indent=4,
                    sort_keys=True,
                )
            f.write("\n")

    cmd = (
        'python infer/modules/train/train.py -e "%s" -sr %s -f0 %s -bs %s -g %s -te %s -se %s %s %s -l %s -c %s -sw %s -v %s'
        % (
            exp_dir1,
            sr2,
            1 if if_f0_3 else 0,
            batch_size12,
            gpus16,
            total_epoch11,
            save_epoch10,
            "-pg %s" % pretrained_G14 if pretrained_G14 != "" else "",
            "-pd %s" % pretrained_D15 if pretrained_D15 != "" else "",
            1 if if_save_latest13 == True else 0,
            1 if if_cache_gpu17 == True else 0,
            1 if if_save_every_weights18 == True else 0,
            version19,
        )
    )
    p = Popen(cmd, shell=True, cwd=now_dir, stdout=PIPE, stderr=STDOUT, bufsize=1, universal_newlines=True)

    for line in p.stdout:
        print(line.strip())

    p.wait()
    return "训练结束, 您可查看控制台训练日志或实验文件夹下的train.log"
%load_ext tensorboard
%tensorboard --logdir ./logs
training_log = click_train(
    model_name,
    '40k',
    True,
    0,
    save_frequency,
    epochs,
    7,
    True,
    'assets/pretrained_v2/f0G40k.pth',
    'assets/pretrained_v2/f0D40k.pth',
    0,
    cache,
    True,
    'v2',
)
print(training_log)

In [ ]:
#@title Либо включаем GUI (Он забанен для пользователей без подписки)
if not 'installed' in locals():
def inf(msg, style, wdth): inf = widgets.Button(description=msg, disabled=True, button_style=style, layout=widgets.Layout(min_width=wdth));display(inf)/n
    %cd /content/
    from google.colab import drive
    drive.mount('/content/drive')
    from IPython.display import clear_output
    import ipywidgets as widgets
    import os
    if not os.path.exists('/content/drive'):
        print("Твой диск не смонтирован. Создание фейка.")
        os.makedirs('/content/drive/MyDrive')
    if not os.path.exists('/content/drive/MyDrive/project-main'):
        !wget https://huggingface.co/Rejekts/project/resolve/main/project-main.zip -O '/content/project-main.zip' && unzip 'project-main.zip' -d /content/drive/MyDrive
    !cd '/content/drive/MyDrive/project-main' && python download_files.py && pip install -r 'requirements.txt'
    !rm /content/project-main.zip
    !rm -r /content/sample_data
    !mkdir -p /content/dataset
    clear_output()
    inf('\u2714 Готово','success', '50px')
tensorboard = True #@param {type:"boolean"}
if tensorboard:
    %load_ext tensorboard
    %tensorboard --logdir ./logs
%cd /content/drive/MyDrive/project-main
!python app.py --colab